In [ ]:
!pip install pyyaml==5.1
# This is the current pytorch version on Colab. Uncomment this if Colab changes its pytorch version
# !pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
!pip install pyyaml==5.1
# This is the current pytorch version on Colab. Uncomment this if Colab changes its pytorch version
# !pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
!pwd

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
n = 458
path ='../terrorist.mp4'
# tracker = cv2.TrackerTLD_create()
tracker = cv2.TrackerKCF_create()
run_once = 0
for j in range(n):

        
  image_path = os.path.join('../frame' + str(j) + '.jpg')
  im = cv2.imread(image_path)
  outputs = predictor(im)

  pred_classes = outputs["instances"].pred_classes.cpu().detach().numpy()
  pred_boxes = outputs["instances"].pred_boxes
  boxes =[]

  # print(list(pred_boxes[0]))
  for i in range(len(outputs["instances"].pred_classes)):
    if(outputs["instances"].pred_classes[i] == 0):
      predi_classes.append(pred_classes[i])
      boxes = (list(pred_boxes)[i].cpu().detach().numpy())
      predi_boxes.append(boxes[0])
      print(boxes)
      start_point = (boxes[0],boxes[1])
      end_point = (boxes[2],boxes[3])

      im = cv2.rectangle(im, start_point, end_point, (0,255,0) , 2)
      cv2_imshow(im)
      key = cv2.waitKey(1) & 0xFF
      if key == ord("q"):
        break
  if(len(boxes) != 0):
    if( run_once == 0):
      bo = boxes
      bo[2] = int(bo[2] - bo[0])
      bo[3] = int(bo[3] - bo[1])
      bo[0] = int(bo[0])
      bo[1] = int(bo[1])
      bo = tuple(bo)
      print("bo", bo)
      ok = tracker.init(im, bo)
      run_once =1
  
  if(run_once == 1):
    ok, bbox = tracker.update(im)
    if ok:
      p1 = (int(bbox[0]), int(bbox[1]))
      p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
      cv2.rectangle(im, p1, p2, (255,0,0), 2, 1)
    else:
      cv2.putText(im, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
      run_once = 0
      
  cv2.imwrite('/content/drive/MyDrive/frames/frame'+str(j)+'.jpg',im)

  


print(predi_classes)
print(boxes)
cv2_imshow(im)